# Projet maison n° 3

In [33]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [34]:
import os
os.chdir ("/Users/andressoto/Kit-Start-Data/Kit-Start-Data/session3/names")

In [35]:
# names us
def df_names_us():
    
    df = pd.DataFrame(columns=['year','name', 'gender', 'births'])

    for i in range(1880,2021):
        df1 = pd.read_csv(('yob{}.txt').format(i), names=['name', 'gender', 'births'])
        df1['year'] = i 
        df = df.append(df1)
        df = df[['year','name', 'gender', 'births']]

    return df

In [36]:
df_names_us()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
31266,2020,Zykell,M,5
31267,2020,Zylus,M,5
31268,2020,Zymari,M,5
31269,2020,Zyn,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [37]:
os.chdir ("/Users/andressoto/Kit-Start-Data/Kit-Start-Data/session3")

In [38]:
def df_names_fr():
    
    # import CSV
    df = pd.read_csv('nat2020.csv',sep=';')
    
    # on renomme les colonnes 
    df = df.rename(columns={'sexe':'gender','preusuel':'name','annais':'year','nombre':'births'})
    
    # ordre des colonnes
    df = df[['year', 'name', 'gender', 'births']]
    
    # conversion des variables 
    df['year']   = df['year'].astype("object")
    df['gender'] = df['gender'].astype("int")
    df['name']   = df['name'].astype("string")
    df['births'] = df['births'].astype("int")
    
    # reformatage de la variable 'gender' 
    df['gender'] = df['gender'].map({1:'M',2:'F'})
    
    # on ne garde que les prénoms de taille >= 2
    df = df[df['name'].str.len()>=2]
    
    # 1ere lettre majuscule
    df['name'] = df['name'].str.capitalize()
    
    # suppression des lignes inutilisables
    df = df[df['year'] != 'XXXX']
    df = df[df['name'] != '_prenoms_rares']
    
    # on trie les valeurs
    df = df.sort_values(
    by = ['year','gender','births','name'],
    ascending = [True,True,False,True])
    
    # reset index
    df = df.reset_index(drop=True)
    
    return df

In [39]:
df_names_fr()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
630401,2020,Élouen,M,3
630402,2020,Énes,M,3
630403,2020,Éphraïm,M,3
630404,2020,Éros,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [44]:
# taux de change
def df_taux_change(devise):
    # Lire le csv, en ajoutant les dates en index, en remplaçant les valeurs manquantes ou '-' par des NaN et en enlevant les 5 premières lignes
    df = pd.read_csv('./Webstat_Export_20210929.csv', sep=';', index_col='Titre :', na_values=['','-'], skiprows=[1,2,3,4,5])

    # Changement du format de l'index
    df.index=pd.to_datetime(df.index,format='%d/%m/%Y')

    # Supprimer les lignes avec des valeurs NaN
    df.dropna(inplace=True)

    # Renommer les colonnes avec les sigles des devises
    df.rename(columns=lambda s: s[-4:-1], inplace=True)
    
    # Conversion du type en float
    df.replace(',','.',regex=True, inplace=True)
    df = df.astype(float)
    
    df = pd.DataFrame(df[devise], columns=devise)

    return df

In [48]:
df_taux_change(['AUD','BGN','CHF'])

,AUD,BGN,CHF
Titre :,,,
2006-12-29,1.6691,1.9558,1.6069
2006-12-28,1.6699,1.9558,1.6058
2006-12-27,1.6765,1.9558,1.6058
2006-12-22,1.6792,1.9558,1.6029
2006-12-21,1.6785,1.9558,1.6032
...,...,...,...
2000-07-25,1.6072,1.9462,1.5538
2000-07-24,1.5905,1.9464,1.5555
2000-07-21,1.5945,1.9465,1.5511


### Tests

In [49]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [50]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [51]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... FAIL
test_df_names_us (__main__.Lesson4Tests) ... ERROR
test_df_taux_change (__main__.Lesson4Tests) ... ok

ERROR: test_df_names_us (__main__.Lesson4Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-49-b976908de6f7>", line 5, in test_df_names_us
    df = df_names_us()
  File "<ipython-input-35-9ce58ff250a4>", line 7, in df_names_us
    df1 = pd.read_csv(('yob{}.txt').format(i), names=['name', 'gender', 'births'])
  File "/Users/andressoto/opt/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 610, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/Users/andressoto/opt/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 462, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/Users/andressoto/opt/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 819, in __init__
    self._engine 